In [1]:
import numpy as np
from numpy import random
import pandas as pd
import scipy.stats as st
from pathlib import Path
import sys
sys.path.append('sobol_seq-0.1-np19py34_0/lib/python3.4/site-packages')
sys.path.append('getInvalid.py')
from sobol_seq import sobol_seq
import random
import getInvalid

: 

: 

**In the following cell, we figure out what the materials are called. For SW to know the material properties, it has to look them up from a specified file location. The formatting of this is difficult to get right, so we save a design study template file from solidworks, then load it here and examine the material names**

In [ ]:
template=pd.read_csv("Walker_Generation_Template.csv", encoding="ISO-8859-1")
df=template.copy()
# print(df)
df.reset_index(inplace=True)  
if "index" in df.columns:
    df.drop(["index"], axis=1, inplace=True)
# print(df)
df.drop(df.index[1], inplace=True)
df.drop(df.columns[0:2], axis=1, inplace=True)

df.columns = df.iloc[0]
# print(df.columns)
df.drop(df.index[0], inplace=True)

#Define specific materials to insert into design templates
steel=" AISI 4130 Steel#^# annealed at 865C@SOLIDWORKS Materials$SOLIDWORKS Materials|AISI 4130 Steel#^# annealed at 865C|94$C:/Program Files/SOLIDWORKS Corp/SOLIDWORKS/lang/english/sldmaterials/SOLIDWORKS Materials.sldmat|AISI 4130 Steel#^# annealed at 865C|94"
aluminum=" 6061-T6 (SS)@SOLIDWORKS Materials$SOLIDWORKS Materials|6061-T6 (SS)|164$C:/Program Files/SOLIDWORKS Corp/SOLIDWORKS/lang/english/sldmaterials/SOLIDWORKS Materials.sldmat|6061-T6 (SS)|164"
titanium=" Ti-6Al-4VSolution treated and aged (SS)@SOLIDWORKS Materials$SOLIDWORKS Materials|Ti-6Al-4VSolution treated and aged (SS)|205$C:/Program Files/SOLIDWORKS Corp/SOLIDWORKS/lang/english/sldmaterials/SOLIDWORKS Materials.sldmat|Ti-6Al-4VSolution treated and aged (SS)|205"
print(steel)
print(aluminum)
print(titanium)
df.drop(df.index, axis=0, inplace=True)
# df.columns


**We set the parameter ranges for the parameters here:**

In [ ]:

scaledf=pd.DataFrame()

# scaledf["Name in Solidworks"] = [minimum,range]

scaledf["H"] = [31, 9]
scaledf["L1"] = [13, 9]
scaledf["L2"] = [5, 30]
scaledf["L3"] = [1, 10]
scaledf["L4"] = [3, 10]
scaledf["L5"] = [9, 20]
scaledf["L7"] = [7, 18]
scaledf["L8"] = [5, 10]
scaledf["A2"] = [100, 75]
scaledf["D"] = [0.05, 0.6]
scaledf["T"] = [0.05, 0.3]
scaledf["D2"] = [0.05, 0.6]
scaledf["T2"] = [0.05, 0.3]
scaledf["D3"] = [0.25, 0.7]
#material
scaledf.index=["min", "factor"]
scaledf=scaledf.transpose()

**We create the sobol sequence an scale the sobol vectors by the scale factors defined in the previous cell.**

In [ ]:
seqlength=55000
sobolvec= sobol_seq.i4_sobol_generate(len(scaledf.index), seqlength)

scalefactor = scaledf["factor"].values
scalemin = scaledf["min"].values
unscaled = np.array(sobolvec)

expL3 = 1
expL4 = 1

scaled = []
for row in unscaled:
    scaled_row = []
    for i, value in enumerate(row):
        if(scaledf.index[i] == "L3"):
            scaled_value = (value**expL3) * scalefactor[i] + scalemin[i]
        elif(scaledf.index[i] == "L4"):
            scaled_value = (value**expL4) * scalefactor[i] + scalemin[i]
        else:
            scaled_value = value * scalefactor[i] + scalemin[i]
        scaled_row.append(scaled_value)
    scaled.append(scaled_row)


# scaled = unscaled * scalefactor + scalemin

scaled = pd.DataFrame(scaled, columns = scaledf.index )

# print(np.shape(res))
# numconfigs=1000
# numparams=len(res[0,:])
# configs=np.zeros((numconfigs,numparams))
# i=0
# sobidx=0
# while i<(numconfigs) and sobidx<seqlength:
#     config=res[sobidx,:]
#     sobidx+=1
#     if checkconfig(config):
#         configs[i,:]=config
#         if config[0]==float("NaN"):
#             print(res[sobidx,:])
#         i+=1
# configdf=pd.DataFrame(configs, columns=cols)
print(scaled)

**We randomize a few parameters that are not continuous variables. For example, we have three discrete materials. We also have two boolean parameters (true/false) governing whether we have seat stay and chain stay bridges on the bike**

In [ ]:
# count=0
walkers=[]
df=scaled.copy()
df.columns = [" " + string for string in list(df.columns)]
        
for idx in df.index:
    matnum = random.randint(0,2)
    if matnum==0:
        df.at[idx, " CrossMat"] = steel
    if matnum==1:
        df.at[idx, " CrossMat"] = aluminum
    if matnum==2:
        df.at[idx, " CrossMat"] = titanium
    matnum = random.randint(0,2)
    if matnum==0:
        df.at[idx, " FrameMat"] = steel
    if matnum==1:
        df.at[idx, " FrameMat"] = aluminum
    if matnum==2:
        df.at[idx, " FrameMat"] = titanium
    # if (df.at[idx, " D2"] + df.at[idx, " T2"]) > (df.at[idx, " D"] + df.at[idx, " T"]): 
    #     df.at[idx, " D2"] = df.at[idx, " D"]
    #     df.at[idx, " T2"] = df.at[idx, " T"]
        
# print(df)


invalid_walkers, perc=getInvalid.getInvalid(df)
print(len(invalid_walkers))
print(invalid_walkers)
df.drop(invalid_walkers, inplace=True)

# print(np.array(bridges))
# np.save("./Batch Info/bridges", np.array(bridges))

**Save the file for reference, then load it back up (avoids having to run the above code every time we generate a batch of designs to simulate). We typically simulate ~200 at a time otherwise SW gets overwhelmed (max 500).**

In [ ]:
df.to_csv("Processed_structural_data_sobol.csv")

In [ ]:
augdf=pd.read_csv("FINALProcessed_structural_data_sobol.csv", index_col=0)

augdf=augdf.iloc[:]

**Using the "Design Study Template" File, we cram the generated designs into the correct format for SW to read, then save**

In [ ]:
# for idx in augdf.index:
#     if augdf.at[idx, " CS F"]==augdf.at[idx, " BB Length"]/2:
#         augdf.at[idx, " CS F"]=augdf.at[idx, " CS F"]-.0001
#         print("Adjusting CS F for model " + str(idx) + " by 0.0001 since it is half of BB length and would cause model to lock!")


template=pd.read_csv("Walker_Generation_Template.csv", encoding="ISO-8859-1")
# if "index" in template.columns:
#     template.drop(["index"], axis=1, inplace=True)
newindex=augdf.columns.insert(0,"Status")
augdf["Status"]=np.full(len(augdf.index), 1)
augdf=augdf[newindex]
augdf.index=["Walker" + str(i) for i in augdf.index]
# print(template)
template.reset_index(inplace=True)  
units=template.iloc[1]

#Old code to drop "Level_0", replace this by dropping entries from top of units until reaching correct length
#     units.drop(["level_0"], axis=0, inplace=True)

while len(units.index)>len(augdf.columns):
    units.drop([units.index[0]], axis=0, inplace=True)
units.index=augdf.columns
units.rename("Units", inplace=True)
augdf=augdf.append(units)
newindices= augdf.index[:-1].insert(0, augdf.index[-1])
augdf=augdf.reindex(newindices)
augdf.columns.rename("Parameters", inplace=True)

augdf=augdf.T.reset_index().T
cols=[len(augdf.index)-2, len(augdf.columns)-1] + [""]*(len(augdf.columns)-2)
# print(cols)
augdf.columns=cols
augdf.columns.rename("Design Set", inplace=True)
# print(units)
print(augdf)

In [ ]:
augdf.to_csv("generated_param_study_template.csv", index_label="Design Set")

Combining and formatting all the files

In [3]:
import numpy as np
from numpy import random
import pandas as pd
import scipy.stats as st
from pathlib import Path
import sys
sys.path.append('sobol_seq-0.1-np19py34_0/lib/python3.4/site-packages')
sys.path.append('getInvalid.py')
from sobol_seq import sobol_seq
import random
import getInvalid

augdf=pd.read_csv("FINALProcessed_structural_data_sobol.csv", index_col=0)

augdf=augdf.iloc[:]
# Read each CSV file and add its elements to the series
columns = ['Calc', 'H', 'L1', 'L2', 'L3', 'L4', 'L5', 'L7', 'L8', 'A2', 'D', 'T', 'D2', 'T2', 'D3', 'CrossMat', 'FrameMat', 'Mass1', 'Minimum Factor of Safety1', 'Center_Y', 'Center_Z', 'Leg_Displacement', 'Handle_X','Handle_Y', 'Handle_Z']

file_names = ["alal0-200.csv", "alal200-400.csv", "alal400-600.csv", 
            "alst0-200.csv", "alst200-400.csv", "alst400-600.csv",
            "alti0-200.csv", "alti200-400.csv", "alti400-600.csv",

            "stal0-200.csv", "stal200-400.csv", "stal400-600.csv",
            "stst0-200.csv", "stst200-400.csv", "stst400-600.csv",
            "stti0-200.csv", "stti200-400.csv", "stti400-600.csv",

            "tial0-200.csv", "tial200-400.csv", "tial400-600.csv",
            "tist0-200.csv", "tist200-400.csv", "tist400-600.csv",
            "titi0-200.csv", "titi200-400.csv", "titi400-600.csv",
            ]
combined_df = pd.DataFrame()

for file_name in file_names:
    df = pd.read_csv(file_name, header=None, skiprows=3)
    df = df.iloc[:, 5:]
    combined_df = pd.concat([combined_df, df], axis=1)
combined_df = combined_df.transpose()
#combined_df.columns = columns
combined_df.columns = columns
combined_df.reset_index(drop=True, inplace=True)
# num_rows_to_copy = min(len(combined_df), len(augdf))

# # Trim 'augdf' columns "CrossMat" and "FrameMat" to fit the size of 'combined_df'
# trimmed_crossmat = augdf[" CrossMat"].iloc[:num_rows_to_copy].reset_index(drop=True)
# trimmed_framemat = augdf[" FrameMat"].iloc[:num_rows_to_copy].reset_index(drop=True)
# # Copy the trimmed columns to 'combined_df'
# combined_df["CrossMat"] = trimmed_crossmat
# combined_df["FrameMat"] = trimmed_framemat
combined_df = combined_df[combined_df.iloc[:, 0] != "Estimated"]
combined_df = combined_df.iloc[:, 1:]
combined_df.to_csv("formattedData.csv")

            Calc         H        L1        L2        L3        L4        L5  \
0     Calculated  35.64063  15.67188  18.59375   2.71875   5.96875   17.4375   
1     Calculated  37.96094  14.33594  25.85938   6.54688  12.60938  21.34375   
2     Calculated  34.86719  17.99219  26.79688   9.35938   9.79688  18.21875   
3     Calculated  34.62109  21.75391  23.39844   5.88281   6.55469  17.04688   
4     Calculated  35.81641  21.68359  24.57031   2.67969   7.57031  20.32813   
...          ...       ...       ...       ...       ...       ...       ...   
5394  Calculated  31.33014  20.72284  26.22009   4.77625   8.79651  11.19116   
5395  Calculated  35.47858  14.60565  30.20447   9.69812   7.46838  12.90991   
5396  Calculated    39.416  16.29315  32.07947   1.57312  10.59338  21.65991   
5397  Calculated  35.33795  20.93378   9.11072   5.47937  10.43713  26.34741   
5398  Calculated   36.1817  17.84003  17.54822  10.16687   9.49963  28.22241   

            L7        L8         A2  ..